In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("CUDA is available. Using GPU.")
else:
  device = torch.device("cpu")
  print("CUDA is not available. Using CPU.")

use_gpu = True

In [ ]:
class MNIST_Convnet(nn.Module):
  def __init__(self):
    super(MNIST_Convnet, self).__init__()
    # input channels should be equal to the amount of color channels in the imaage
    # for mnist, this is 1 because it is greyscale
    # for cifar-10, it should be 3 for the red, green and blue channels
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    # to calculate the amount of nodes for the first layer of the neural net
    # we take the original size (28 x 28) divided by the stride of the first pooling layer = 14 * 14
    # the 14 * 14 output is again divided by 2, because of the stride of 2 in the second pooling layer = 7 * 7
    # to calculate the amount of nodes, this has to be multiplied by the depth of the last convolutional layer
    # with a depth of 64, there would be 7 * 7 * 64 = 3136 nodes
    self.fc1 = nn.Linear(64 * 7 * 7, 128)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    # -1 is the batch size, which is automatically done based on the data
    # 1 stands for the depth, which is 1 for grey-scaled
    # 28 stands for the dimensions of the images (since the image is 28 by 28)
    x = x.view(-1, 1, 28, 28)
    x = self.relu(self.conv1(x))
    x = self.pool1(x)

    x = self.relu(self.conv2(x))
    x = self.pool2(x)

    # flatten the outputs to be able to go into the fc layer(s)
    # it should be flattened the same dimensions as the amount of nodes in the fc layer (like explained above)
    x = x.view(-1, 64 * 7 * 7)
    x = self.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [ ]:
epochs = 10
batch_size = 64
num_output_classes = 10
learning_Rate = 0.001
use_gpu = True

In [ ]:
mnist_train = pd.read_csv('./data/mnist/mnist_train.csv', header=None)
mnist_test = pd.read_csv('./data/mnist/mnist_test.csv', header=None)

mnist_train_x = mnist_train.iloc[:, 1:].values / 255.0
mnist_train_y = mnist_train.iloc[:, 0].values 
mnist_test_x = mnist_test.iloc[:, 1:].values / 255.0
mnist_test_y = mnist_test.iloc[:, 0].values

if use_gpu:
  mnist_train_x = torch.tensor(mnist_train_x, dtype=torch.float32).to(device)
  mnist_train_y = torch.tensor(mnist_train_y, dtype=torch.long).to(device)
  mnist_test_x = torch.tensor(mnist_test_x, dtype=torch.float32).to(device)
  mnist_test_y = torch.tensor(mnist_test_y, dtype=torch.long).to(device)
else:
  mnist_train_x = torch.tensor(mnist_train_x, dtype=torch.float32)
  mnist_train_y = torch.tensor(mnist_train_y, dtype=torch.long)
  mnist_test_x = torch.tensor(mnist_test_x, dtype=torch.float32)
  mnist_test_y = torch.tensor(mnist_test_y, dtype=torch.long)


train_dataset = TensorDataset(mnist_train_x, mnist_train_y)
test_dataset = TensorDataset(mnist_test_x, mnist_test_y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

mnist_train_x.is_cuda


In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs):
  print(f"Training on {'GPU :)' if next(model.parameters()).is_cuda else 'CPU.. :('}\n")
  for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
            
      _, predicted = torch.max(outputs, 1)
      correct += (predicted == labels).sum().item()
      total += labels.size(0)
        
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")
        
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      for images, labels in test_loader:
          outputs = model(images)
          loss = criterion(outputs, labels)
          test_loss += loss.item()
          _, predicted = torch.max(outputs, 1)
          correct += (predicted == labels).sum().item()
          total += labels.size(0)
        
    test_accuracy = 100 * correct / total
    print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")
    print("=========================================")
    print()

In [ ]:
if use_gpu:
  model = MNIST_Convnet().to(device)
else:
  model = MNIST_Convnet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_Rate)


# cpu runtime: 436 seconds
# gpu runtime: 42.6 seconds
train_model(model, train_loader, test_loader, criterion, optimizer, 10)